## Loading Libraries and establishing connection to the database

In [3]:
import pymysql
import pandas as pd
import yfinance as yf

conn = pymysql.connect(host = '130.211.219.71',port=int(3306),user='root',passwd='Nodal@987',db='nodalDB')
cur = conn.cursor()

## Nifty and Banknifty stocks

In [2]:
nifty = pd.read_csv("nifty.csv")
banknifty = pd.read_csv("banknifty.csv")

In [3]:
nifty_list = nifty['Symbol'].tolist()
banknifty_list = banknifty['Symbol'].tolist()

In [4]:
print(nifty_list)
print(len(nifty_list))
print(banknifty_list)
print(len(banknifty_list))
final_list = list(sorted(set(nifty_list) | set(banknifty_list)))
print(final_list)
print(len(final_list))

['ADANIPORTS', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO', 'BAJFINANCE', 'BAJAJFINSV', 'BPCL', 'BHARTIARTL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'GRASIM', 'HCLTECH', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'HDFC', 'ICICIBANK', 'ITC', 'IOC', 'INDUSINDBK', 'INFY', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'M&M', 'MARUTI', 'NTPC', 'NESTLEIND', 'ONGC', 'POWERGRID', 'RELIANCE', 'SBILIFE', 'SHREECEM', 'SBIN', 'SUNPHARMA', 'TCS', 'TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TECHM', 'TITAN', 'UPL', 'ULTRACEMCO', 'WIPRO']
50
['AUBANK', 'AXISBANK', 'BANDHANBNK', 'FEDERALBNK', 'HDFCBANK', 'ICICIBANK', 'IDFCFIRSTB', 'INDUSINDBK', 'KOTAKBANK', 'PNB', 'RBLBANK', 'SBIN']
12
['ADANIPORTS', 'ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BHARTIARTL', 'BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK', 'GRASIM', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO'

## Function to add data to the database table

In [ ]:
from time import gmtime, strftime

def add_data(stock_name = "", start_date = "", end_date = ""):
    name = stock_name + ".NS"
    stock = yf.download(name, start=start_date, end=end_date, progress=False,)
    stock.dropna(inplace = True)
    stock.reset_index(inplace = True)
    stock.rename(columns = {'Date':'trading_date', 'Open' : 'open', 'High' : 'high','Low' : 'low','Close' : 'close', 'Adj Close' : 'adj_close', 'Volume' : 'volume'}, inplace = True)
    stock_name = stock_name
    stock['stock_name'] = [stock_name] * stock.shape[0]
    stock['timestamp'] = [strftime("%Y-%m-%d %H:%M:%S", gmtime())] * stock.shape[0]
    
    for i,row in stock.iterrows():
        sql = """INSERT INTO nifty_banknifty_stocks_list(trading_date, open, high, low, close, adj_close, volume, stock_name, timestamp) 
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        cur.execute(sql,tuple(row))
        print("Record Added", i)
    conn.commit()

In [ ]:
for stocks in final_list:
    print("Trying to add {} in the database".format(stocks))
    add_data(stock_name = stocks,start_date = '2015-01-01',end_date = '2021-11-01')
    print(stocks, "is added successfully to the database.")

## Fetching the data from the database table

In [4]:
df = pd.read_sql_query("SELECT * FROM nifty_banknifty_stocks_list ",conn)
df.head()

,id,trading_date,open,high,low,close,adj_close,volume,stock_name,timestamp
0,57,2015-01-01 00:00:00,319,322.5,316.25,319.54998779296875,308.71337890625,1456204,ADANIPORTS,2021-12-29 00:08:44
1,58,2015-01-02 00:00:00,319.3500061035156,325.79998779296875,318.04998779296875,319.3500061035156,308.5201721191406,2894058,ADANIPORTS,2021-12-29 00:08:44
2,59,2015-01-05 00:00:00,320.45001220703125,327.5,319.3500061035156,323.79998779296875,312.81927490234375,2099786,ADANIPORTS,2021-12-29 00:08:44
3,60,2015-01-06 00:00:00,321.6499938964844,331.45001220703125,315.6000061035156,321.8500061035156,310.9354248046875,3672197,ADANIPORTS,2021-12-29 00:08:44
4,61,2015-01-07 00:00:00,321.95001220703125,328.70001220703125,317.3999938964844,321.1000061035156,310.2108154296875,2981544,ADANIPORTS,2021-12-29 00:08:44


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90989 entries, 0 to 90988
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            90989 non-null  int64 
 1   trading_date  90989 non-null  object
 2   open          90989 non-null  object
 3   high          90989 non-null  object
 4   low           90989 non-null  object
 5   close         90989 non-null  object
 6   adj_close     90989 non-null  object
 7   volume        90989 non-null  object
 8   stock_name    90989 non-null  object
 9   timestamp     90989 non-null  object
dtypes: int64(1), object(9)
memory usage: 6.9+ MB


In [6]:
df.to_csv("nifty_banknifty_stocks_list.csv")